In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
max_seq_length = 768 
lora_rank = 64

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-1.5B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,
    full_finetuning=False
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/ziyuan/final_project/unsloth/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-01 12:50:41 [__init__.py:239] Automatically detected platform cuda.


2025-05-01 12:50:42,228	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.2.
   \\   /|    NVIDIA GeForce RTX 4070 Ti. Num GPUs = 1. Max memory: 11.994 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-1.5b-instruct-unsloth-bnb-4bit with actual GPU utilization = 44.81%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 11.99 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 768. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 4.04 GB. Also swap space = 2 GB.
INFO 05-01 12:50:50 [config.py:585] This model supports multiple tasks: {'generate', 'reward', 'classify', 'score', 'embed'}. Defaulting to 'generate'.
WARNING 05-01 12:50:5

...

INFO 05-01 12:50:55 [punica_selector.py:18] Using PunicaWrapperGPU.


INFO 05-01 12:50:55 [model_runner.py:1146] Model loading took 1.5708 GB and 4.016843 seconds
INFO 05-01 12:50:56 [worker.py:267] Memory profiling takes 0.97 seconds
INFO 05-01 12:50:56 [worker.py:267] the current vLLM instance can use total_gpu_memory (11.99GiB) x gpu_memory_utilization (0.45) = 5.37GiB
INFO 05-01 12:50:56 [worker.py:267] model weights take 1.57GiB; non_torch_memory takes 0.04GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 2.71GiB.
INFO 05-01 12:50:56 [executor_base.py:111] # cuda blocks: 6349, # CPU blocks: 4681
INFO 05-01 12:50:56 [executor_base.py:116] Maximum concurrency for 768 tokens per request: 132.27x
INFO 05-01 12:50:57 [model_runner.py:1442] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasi

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:16<00:00,  1.61it/s]

INFO 05-01 12:51:13 [model_runner.py:1570] Graph capturing finished in 18 secs, took 0.41 GiB
INFO 05-01 12:51:13 [llm_engine.py:447] init engine (profile, create kv cache, warmup model) took 18.42 seconds



Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
import json
from datasets import Dataset

# --- Constants ---
NUM_SAMPLES_TO_LOAD = 612 
TEST_SIZE = 100 
SEED = 42 

# --- Modified Data Loading Function for SFT Only ---
def load_and_format_data_for_sft(json_path="./data.json", tokenizer=None, num_samples=NUM_SAMPLES_TO_LOAD) -> Dataset:
    """Loads the first 'num_samples' data points and formats them for SFT.

    Args:
        json_path (str): Path to the JSON data file.
        tokenizer: The tokenizer instance.
        num_samples (int): The number of samples to load from the beginning of the file.

    Returns:
        sft_dataset: Dataset formatted for SFT (fields: 'text', 'original_index').
    """

    with open(json_path, 'r', encoding='utf-8') as f:
        original_data = json.load(f)

    data_to_process = original_data[:num_samples]
    print(f"Loading the first {len(data_to_process)} entries from {json_path} (requested: {num_samples})...")

    sft_formatted_list = []
    skipped_count = 0 

    for idx, entry in enumerate(data_to_process): 
        prompt_content = entry.get('prompt')
        raw_response_content = entry.get('response')

        if not isinstance(prompt_content, list):
             print(f"Warning: prompt_content at index {idx} is not a list. Skipping.")
             skipped_count += 1
             continue 

        if not raw_response_content:
             print(f"Warning: Missing or empty 'response' at index {idx}. Skipping.")
             skipped_count += 1
             continue

        sft_messages = prompt_content + [{'role': 'assistant', 'content': raw_response_content}]
        try:
            sft_formatted_text = tokenizer.apply_chat_template(
                sft_messages,
                tokenize=False,
                add_generation_prompt=False
            )
        except Exception as e:
            print(f"Error applying chat template for SFT at index {idx}: {e}")
            skipped_count += 1
            continue

        sft_entry = {'text': sft_formatted_text, 'original_index': idx}
        sft_formatted_list.append(sft_entry)

    sft_dataset = Dataset.from_list(sft_formatted_list)

    print(f"\nFinished processing for SFT.")
    print(f"Total entries processed from the subset: {len(data_to_process)}")
    print(f"Entries skipped during processing: {skipped_count}")
    print(f"Final SFT dataset size: {len(sft_dataset)}")

    return sft_dataset

dataset_path = "./data.json"
sft_dataset_full = load_and_format_data_for_sft(dataset_path, tokenizer)

if len(sft_dataset_full) == 0:
    raise ValueError("SFT dataset is empty after loading and filtering. Cannot train.")


actual_test_size = min(TEST_SIZE, len(sft_dataset_full))
if actual_test_size < TEST_SIZE:
     print(f"Warning: Requested test_size ({TEST_SIZE}) is larger than the loaded dataset size ({len(sft_dataset_full)}). Using test_size={actual_test_size}.")

if len(sft_dataset_full) <= actual_test_size:
     raise ValueError(f"Dataset size ({len(sft_dataset_full)}) is not large enough for the test split size ({actual_test_size}). Need at least {actual_test_size + 1} samples.")

print(f"\nSplitting SFT dataset (Total: {len(sft_dataset_full)}) with test_size={actual_test_size} and shuffle=False...")

split_sft_dataset = sft_dataset_full.train_test_split(
    train_size=512,
    test_size=actual_test_size,
    seed=SEED, 
    shuffle=False
)
train_sft_dataset = split_sft_dataset['train']
eval_sft_dataset_for_loss = split_sft_dataset['test'] 

eval_indices = sorted(list(eval_sft_dataset_for_loss['original_index']))
print(f"Indices selected for SFT evaluation split (should be the last {len(eval_indices)} indices): {eval_indices[:5]}...{eval_indices[-5:]}") 

print(f"\nSFT Dataset split:")
print(f"  Training set size (SFT): {len(train_sft_dataset)}")
print(f"  Evaluation set size (SFT Loss): {len(eval_sft_dataset_for_loss)}")

train_dataset = train_sft_dataset 


Loading the first 612 entries from ./data.json (requested: 612)...

Finished processing for SFT.
Total entries processed from the subset: 612
Entries skipped during processing: 0
Final SFT dataset size: 612

Splitting SFT dataset (Total: 612) with test_size=100 and shuffle=False...
Indices selected for SFT evaluation split (should be the last 100 indices): [512, 513, 514, 515, 516]...[607, 608, 609, 610, 611]

SFT Dataset split:
  Training set size (SFT): 512
  Evaluation set size (SFT Loss): 100


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [3]:
from transformers import TrainingArguments

if 'bf16_supported' not in globals():
    bf16_supported = is_bfloat16_supported()

training_args = TrainingArguments(
    per_device_train_batch_size = 16, 
    gradient_accumulation_steps = 1, 
    warmup_ratio = 0.1, 
    max_grad_norm = 1.0,
    weight_decay=0.01,
    num_train_epochs = 3, 
    learning_rate = 5e-4, 
    logging_steps = 5,    
    optim = "adamw_8bit", 
    save_strategy = "steps", 
    save_steps = 8,       
    eval_strategy = "steps", 
    eval_steps = 8,       
    bf16 = bf16_supported,
    fp16 = not bf16_supported,
    output_dir = "outputs_sft", 
    report_to = "none",
    seed = 3407,
)

print("\nTrainingArguments set for SFT.")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")



TrainingArguments set for SFT.
Effective batch size: 16


In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_sft_dataset_for_loss,
    args = training_args,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
)

print("\nStarting SFT training...")
trainer.train()
print("\nSFT training finished.")

model.save_lora("sft_final_lora")


Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 100/100 [00:00<00:00, 184.28 examples/s]



Starting SFT training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 512 | Num Epochs = 3 | Total steps = 96
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 73,859,072/5,000,000,000 (1.48% trained)


Step,Training Loss,Validation Loss
8,1.221200,0.209433
16,0.162600,0.162517
24,0.170900,0.147904
32,0.140100,0.142871
40,0.125300,0.140737
48,0.124100,0.137847
56,0.122400,0.138920
64,0.124100,0.135619
72,0.107700,0.135125
80,0.110900,0.134143


Unsloth: Will smartly offload gradients to save VRAM!

SFT training finished.
